# Vector DB Question/Answering

In [1]:
# langchain dependencies
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain import VectorDBQA, VectorDBQAWithSourcesChain

# steamship_langchain dependencies
from steamship import Steamship
from steamship_langchain import OpenAI
from steamship_langchain.vectorstores import SteamshipVectorStore

In [2]:
client = Steamship()

In [4]:
with open("../../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [7]:
docsearch = SteamshipVectorStore.from_texts(
    client=client,
    texts=texts,
    embedding="text-embedding-ada-002",
    index_name="example-index",
    metadatas=[{"source": i} for i in range(len(texts))],
)

# Creating chains with VectorDBQA.from_chain_type

Steamship's vectorstore support all 4 chain types to create a VectorDBQA chain. 
For a more detailed walkthrough of these types, please see this notebook.

In [8]:
qa = VectorDBQA.from_chain_type(
    llm=OpenAI(client=client),
    chain_type="stuff",  # or map_reduce
    vectorstore=docsearch,
    return_source_documents=True,
)

In [9]:
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"query": query})

In [10]:
result["result"]

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds and a consensus builder who has received broad support, including from the Fraternal Order of Police and former judges appointed by Democrats and Republicans."

In [11]:
result["source_documents"]

[Document(page_content='As I did four days ago, I’ve nominated a Circuit Court of Appeals — Ketanji Brown Jackson. One of our nation’s top legal minds who will continue in just Brey- — Justice Breyer’s legacy of excellence. A former top litigator in private practice, a former federal public defender from a family of public-school educators and police officers — she’s a consensus builder.\n\nSince she’s been nominated, she’s received a broad range of support, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans.\n\nFolks, if we are to advance liberty and justice, we need to secure our border and fix the immigration system.\n\nAnd as you might guess, I think we can do both. At our border, we’ve installed new technology, like cutting-edge scanners, to better detect drug smuggling.\n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.', lookup_str='', metadata={'source': 39}, lookup_index=0),
 Document(page_cont

## Creating chains with load_qa_chain and combine_documents_chain

In [12]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(OpenAI(client=client, temperature=0), chain_type="stuff")
qa = VectorDBQA(combine_documents_chain=qa_chain, vectorstore=docsearch)

In [13]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a consensus builder, and has received a broad range of support, including from the Fraternal Order of Police and former judges appointed by Democrats and Republicans."

## Creating chains with VectorDBQAWithSourcesChain.from_chain_type

In [14]:
chain = VectorDBQAWithSourcesChain.from_chain_type(
    OpenAI(client=client, temperature=0), chain_type="stuff", vectorstore=docsearch
)

In [15]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': " The president thanked Justice Breyer for his service and mentioned that he nominated Ketanji Brown Jackson to continue in Justice Breyer's legacy of excellence.\n",
 'sources': '38, 39'}